In [1]:
import numpy as np  #
import pandas as pd  #
from datetime import datetime, date, timedelta
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
# from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge,Lasso,ElasticNet
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,RandomForestClassifier
# from sklearn.feature_selection import mutual_info_regression
# from sklearn.svm import SVR, LinearSVC
# from sklearn.pipeline import make_pipeline,Pipeline
# from sklearn.preprocessing import RobustScaler, LabelEncoder, StandardScaler,MinMaxScaler
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import time
from itertools import product
import datetime as dt
import calendar
import gc
from datetime import date, timedelta
import paddle
import paddle.fluid as fluid
import paddle.fluid.dygraph as dygraph
from paddle.fluid.dygraph import Linear
import random

In [2]:
!pip install lightgbm -i https://mirrors.aliyun.com/pypi/simple/
!pip install xgboost

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirror.baidu.com/pypi/simple/


In [3]:
import numpy as np  # linear algebra
import pandas as pd  #
from datetime import datetime, date, timedelta
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge,Lasso,ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.svm import SVR, LinearSVC
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from xgboost import XGBRegressor
import xgboost as xgb

In [7]:
import pickle
!pip install --upgrade pandas

Looking in indexes: https://mirror.baidu.com/pypi/simple/
Requirement already up-to-date: pandas in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (1.2.3)


In [8]:
import pandas as pd  #

In [9]:
PATH = './data/data71980/'
train = pd.read_csv(PATH + 'train.csv')
# train = pd.read_csv('./data/data19383/train.csv', usecols=[2, 3, 4, 6, 7, 18])
# set index to ID to avoid droping it later
# 把测试集的id列作为索引，防止误删
test  = pd.read_csv(PATH + 'submission.csv').set_index('customer_id')

In [10]:
X_train = pd.read_pickle('data/data71980/X_train.pkl')
X_val = pd.read_pickle('data/data71980/X_val.pkl')
X_test = pd.read_pickle('data/data71980/X_test.pkl')

In [11]:
import numpy as np

In [12]:
y_train = np.load("data/data71980/y_train.npy")
y_val = np.load("data/data71980/y_val.npy")

In [13]:
import gc

In [14]:
X_l, y_l = [], []
X_l.append(X_train)
y_l.append(y_train)
X_l.append(X_val)
y_l.append(y_val)
X_train_cv = pd.concat(X_l, axis=0)
y_train_cv = np.concatenate(y_l, axis=0)
del X_l
del y_l
gc.collect()
X_train_cv.to_pickle('work/X_train_cv.pkl')
np.save("work/y_train_cv.npy", y_train_cv)

In [15]:
X_train_cv = pd.read_pickle('work/X_train_cv.pkl')
y_train_cv = np.load("work/y_train_cv.npy")
X_test = pd.read_pickle('data/data71980/X_test.pkl')

In [16]:
from xgboost import XGBRegressor
import xgboost as xgb

In [17]:
xgb_train = xgb.DMatrix(X_train_cv.values, label=y_train_cv)
xgb_test = xgb.DMatrix(X_test.values)

params = {'booster': 'gbtree',
          'objective': 'rank:pairwise',  # 二分类的问题
          # 'gamma':0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
          'max_depth': 4,  # 构建树的深度，越大越容易过拟合
          # 'lambda':2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
          'subsample': 0.8,  # 随机采样训练样本
          'colsample_bytree': 0.8,  # 生成树时进行的列采样
          'min_child_weight': 2,
          # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
          # ，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
          # 这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
          'scale_pos_weight': 100,
          'silent': 0,  # 设置成1则没有运行信息输出，最好是设置为0.
          'eta': 0.03,  # 如同学习率
          'nthread': 8,  # cpu 线程数
          'eval_metric': 'logloss'  # 评价方式
          }

plst = list(params.items())
num_rounds = 30  # 迭代次数
watchlist = [(xgb_train, 'train')]
# early_stopping_rounds    当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
model = xgb.train(plst, xgb_train, num_rounds, watchlist)
pred_value = model.predict(xgb_test)

[14:58:57] WARNING: ../src/learner.cc:541: 
Parameters: { scale_pos_weight, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.67855
[1]	train-logloss:0.66621
[2]	train-logloss:0.65160
[3]	train-logloss:0.64289
[4]	train-logloss:0.63305
[5]	train-logloss:0.62306
[6]	train-logloss:0.61380
[7]	train-logloss:0.60512
[8]	train-logloss:0.59776
[9]	train-logloss:0.59080
[10]	train-logloss:0.58355
[11]	train-logloss:0.57783
[12]	train-logloss:0.57207
[13]	train-logloss:0.56747
[14]	train-logloss:0.56337
[15]	train-logloss:0.56034
[16]	train-logloss:0.55662
[17]	train-logloss:0.55452
[18]	train-logloss:0.55288
[19]	train-logloss:0.55135
[20]	train-logloss:0.55270
[21]	train-logloss:0.60194
[22]	train-logloss:0.60932
[23]	train-logloss:0.61333
[24]	train-logloss

In [18]:
print(len(pred_value))
print(pred_value)

484245
[0.7536372  0.231422   0.6275625  ... 0.9184368  0.9312559  0.92893404]


In [19]:
sub = pd.read_pickle('data/data71980/X_test.pkl')
print(sub)

        customer_id  mean_7  median_7  max_7  sum_7  mean_7_2  median_7_2  \
0           1000014     0.0       0.0    0.0    0.0       0.0         0.0   
1           1000046     0.0       0.0    0.0    0.0       0.0         0.0   
2           1000069     0.0       0.0    0.0    0.0       0.0         0.0   
3           1000105     0.0       0.0    0.0    0.0       0.0         0.0   
4           1000109     0.0       0.0    0.0    0.0       0.0         0.0   
...             ...     ...       ...    ...    ...       ...         ...   
484240      2826556     0.0       0.0    0.0    0.0       0.0         0.0   
484241      2826558     0.0       0.0    0.0    0.0       0.0         0.0   
484242      2826562     0.0       0.0    0.0    0.0       0.0         0.0   
484243      2826570     0.0       0.0    0.0    0.0       0.0         0.0   
484244      2826574     0.0       0.0    0.0    0.0       0.0         0.0   

        max_7_2  has_sales_days_in_last_7  last_has_sales_day_in_last_7  ..

In [20]:
subb = sub['customer_id']

In [21]:
print(subb)

0         1000014
1         1000046
2         1000069
3         1000105
4         1000109
           ...   
484240    2826556
484241    2826558
484242    2826562
484243    2826570
484244    2826574
Name: customer_id, Length: 484245, dtype: uint32


In [24]:
df_preds = pd.DataFrame(
{    "customer_id": subb, 
    "pred": pred_value}
)

In [26]:
df_preds.head()

AttributeError: 'NotebookFormatter' object has no attribute 'get_result'

   customer_id      pred
0      1000014  0.753637
1      1000046  0.231422
2      1000069  0.627563
3      1000105  0.800692
4      1000109  0.410944

In [27]:
sub = pd.read_csv('data/data71980/submission.csv')

In [28]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1585986 entries, 0 to 1585985
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   customer_id  1585986 non-null  int64  
 1   result       1585986 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 24.2 MB


In [29]:
submission = pd.merge(sub, df_preds, on='customer_id', how='left')
submission.fillna(0,inplace=True)
submission = submission[['customer_id','pred']]
submission.rename(columns={'customer_id':'customer_id','pred':'result'}, inplace=True)

In [30]:
submission.head()

AttributeError: 'NotebookFormatter' object has no attribute 'get_result'

   customer_id    result
0      1000000  0.000000
1      1000014  0.753637
2      1000034  0.000000
3      1000046  0.231422
4      1000048  0.000000

In [31]:
# 将概率值转换为用户是否购买的标签
def f(x):
    if x <= 0.3:
        return 0
    if x > 0.3:
        return 1
    return x

In [32]:
submission['result'] = submission['result'].map(f)

In [26]:
submission.to_csv('submission.csv',index=False)
submission.head()

,customer_id,result
0,1000000,0
1,1000014,1
2,1000034,0
3,1000046,0
4,1000048,0
